# In this notebook, we prototype our data collection process.

Generally speaking, we will exploit the fact that 

In [ ]:
# /r/nottheonion: 1224968000
# /r/TheOnion: 1206303030

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests
from datetime import date
import time

import pickle
import os
import sys

In [28]:
def save_obj(obj, filename):
    with open(filename + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(filename + '.pkl', 'rb') as f:
        return pickle.load(f)

In [89]:
DIR = "~/DSI/Projects/project_3/data/"
FILE1 = "theonion"
FILE2 = "nottheonion"

In [1]:
import time
import requests

class JsonCollector:
    def __init__(self, mintime, subreddit):
        self.url = "https://api.pushshift.io/reddit/search/submission/"
        self.subreddit = subreddit
        # Contains tuples, denoting the ranges in which 
        # we pulled 500 entries, implying we missed some.
        self.gaps=[]
        
        # The amount of time we scrape for submissions each time
        self.timegap = 100000
        
        # The number of consecutive scrapes we received less than 250 submissions.
        self.scrapes_since_gap = 0
        
        # the Unix times we have already searched.
        self.max = time.time()
        self.min = time.time()
        
        # the Unix time before which there cannot be submissions.
        self.MINCUTOFF = mintime
        
        #contains dictionary keyed on postid, with value=json block
        self.data = {}
    
    
    def fetch_json(self, lower, upper):
        r = requests.get(
            self.url,
            params={
                "subreddit": self.subreddit,
                "before": upper,
                "after": lower,
                "size": 500
            }
        )
        assert r.status_code == 200
        return r.json()["data"]
    
    # Find posts starting at max time, up to the current time
    def read_forward(self):
        newtime = min(self.max + self.timegap, time.time() - self.max)
        jsonlist = self.fetch_json(self.max, newtime)        
        
        #Check for gaps; if gap, mark it and half the search window.
        if len(jsonlist) == 500:
            self.gaps.append((self.max, newtime))
            self.scrapes_since_gap = 0
            self.timegap /= 2
        elif len(jsonlist) <= 250:
            if self.scrapes_since_gap >= 2:
                self.timegap *= 2
                self.scrapes_since_gap = 0
            else:
                self.scrapes_since_gap += 1
        # save the data    
        for i in range(len(jsonlist)):
            self.data[jsonlist[i]["id"]] = jsonlist[i]
        self.fill_gaps()
        
    
        
    # Find posts going backwards in time from the oldest we found
    def read_back(self):
        if self.min > self.MINCUTOFF:
            newtime = max(self.min - self.timegap, self.MINCUTOFF)
            jsonlist = self.fetch_json(newtime, self.min)
            #Check for gaps; if gap, mark it.
            if len(jsonlist) == 500:
                self.gaps.append((self.max, newtime))
                self.scrapes_since_gap = 0
                self.timegap /= 2
            elif len(jsonlist) <= 250:
                if self.scrapes_since_gap >= 2:
                    self.timegap *= 2
                    self.scrapes_since_gap = 0
                else:
                    self.scrapes_since_gap += 1
            # save the data    
            for i in range(len(jsonlist)):
                self.data[jsonlist[i]["id"]] = jsonlist[i]
                
        else:
            print("We've already plumbed the depths of this subreddit!")
        pass
        self.fill_gaps()
        
        
    # For segments with gaps, recursively partition the space until there are no gaps.
    def fill_gaps(self):
        for gap in self.gaps:
            middle = (gap[0] + gap[1])/2
            
            # first check the second half
            jsonlist_upper = self.fetch_json(middle, gap[1])
            
        #Check for gaps; if gap, mark it.
        if len(r.json()['data']) == 500:
            self.gaps.append((self.max, newtime))
            self.scrapes_since_gap = 0
        else:
            self.scrapes_since_gap += 1
        
        
        
            

IndentationError: expected an indented block (<ipython-input-1-a57477d6c7c3>, line 31)

In [ ]:
def get_nottheonion():
    pass

In [ ]:
if os.path.exists("data/" + FILE1):
    dict1 = load_obj(FILE1)
else:
    dict1 = {}
    


In [5]:
url = "https://api.pushshift.io/"
url_submissions = "https://api.pushshift.io/reddit/search/submission/"

In [75]:
# The goal is to build a local collection of reddit data that can be trivially expanded if necessary
# but otherwise does not require network access

In [6]:
r = requests.get(
    url_submissions,
    params={
        "subreddit":"theonion",
        "before":1560000000,
        "after": 1559000000,
        "size":500
#         "function": "TIME_SERIES_DAILY",
#         "symbol": "DPZ",
#         "outputsize": "full",
#         "apikey": key
    }
)

In [22]:
display(r.status_code)
display(len(r.json()['data']))
sys.getsizeof(r.json()['data'][10])

200

110

2280

In [16]:
jsonlist = r.json()['data']
for i in range(len(jsonlist)):
    print(jsonlist[i]["id"],jsonlist[i])

btu4xo {'all_awardings': [], 'author': 'DanishPastry789', 'author_flair_css_class': None, 'author_flair_richtext': [], 'author_flair_text': None, 'author_flair_type': 'text', 'author_fullname': 't2_1oyt4bwb', 'author_patreon_flair': False, 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1559007823, 'domain': 'sports.theonion.com', 'full_link': 'https://www.reddit.com/r/TheOnion/comments/btu4xo/depressed_lebron_james_realizes_that_at_his_age/', 'gildings': {}, 'id': 'btu4xo', 'is_crosspostable': True, 'is_meta': False, 'is_original_content': False, 'is_reddit_media_domain': False, 'is_robot_indexable': True, 'is_self': False, 'is_video': False, 'link_flair_background_color': '', 'link_flair_richtext': [], 'link_flair_text_color': 'dark', 'link_flair_type': 'text', 'locked': False, 'media_only': False, 'no_follow': True, 'num_comments': 3, 'num_crossposts': 0, 'over_18': False, 'parent_whitelist_status': 'all_ads', 'permalink': '/r/TheOnion/comments/btu4xo/depressed_lebron_j

In [54]:
len(r.json()['data'])

500